# Heuristics - CVRP

In [16]:
from itertools import cycle

import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib as mpl
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import functions as f
import importlib

## Instantiate data

In [17]:
dataset = pd.read_csv("./data/custom_19.csv", index_col=0)
coordinates = dataset.loc[:, ["x", "y"]]

In [18]:
n_vehicles = 4
seniority_multiplier = [1, 1.2, 1.6, 1]
start_locations = [0, 2, 3, 5]
end_locations = start_locations

# set timeslots for the start locations to zero
for home_indx in start_locations:
    dataset.loc[home_indx, "timeslot"] = np.nan
    dataset.loc[home_indx, "job_type"] = "start"

allow_arbitrary_end_locations = True  # when True, the end locations are not fixed
# this is done by adding a dummy node to the graph that has zero travel time to all locations

if allow_arbitrary_end_locations:
    end_locations = [len(coordinates)] * n_vehicles
    arbitrary_end_location = pd.DataFrame(
        {"x": {len(coordinates): 99}, "y": {len(coordinates): 99}}
    )
    coordinates = pd.concat([coordinates, arbitrary_end_location], axis=0)

N = coordinates.shape[0]

In [19]:
# time windows, each each number represents 1 minute
time_window_dict = {
    "AD": [0, 510],  # 08:00 - 16:30
    "AM": [0, 240],  # 08:00 - 12:00
    "PM": [240, 510],  # 12:00 - 16:30
}

service_time_dict = {
    "start": 0,
    "DF-M Commision": 50,
    "DF-M Exchange": 100,
}

In [20]:
time_matrix_multiplier = 1  # used to make the drive times fill the day more

# assume travel time is proportional to euclidean distance
time_matrix = squareform(pdist(coordinates, metric="euclidean"))

# reduce the drive times to make feasible solutions more likely, will adjust later!
time_matrix = np.round(time_matrix * time_matrix_multiplier, decimals=0).astype(int)

if allow_arbitrary_end_locations:
    time_matrix[:, -1] = time_matrix[-1, :] = 0

    # need to add in the last location to the dataset for the time callback
    dataset.loc[len(dataset)] = [np.nan, np.nan, np.nan, "start"]

## Model

In [21]:
# Create the routing index manager: number of nodes, number of vehicles, depot node
manager = pywrapcp.RoutingIndexManager(N, n_vehicles, start_locations, end_locations)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

### Parameters

In [22]:
# # Same valid for any callback related to arcs/edges
# def time_callback(from_index, to_index):
#     from_node = manager.IndexToNode(from_index)
#     to_node = manager.IndexToNode(to_index)

#     # vehicle_id = routing.VehicleVar(from_index).Value()

#     travel_time = time_matrix[from_node, to_node]

#     service_time = int(service_time_dict[dataset.loc[from_node, "job_type"]]) # * seniority_multiplier[vehicle_id])

#     return (
#         travel_time
#     )
#     # convention is that time is added upon leaving the previous node
#     # return time_matrix[from_node, to_node] + service_time_dict[
#     #     dataset.loc[from_node, "job_type"]
#     #     ]


# transit_callback_index = routing.RegisterTransitCallback(time_callback)

In [23]:
def create_time_callback(vehicle_id):
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        travel_time = time_matrix[from_node, to_node]

        service_time = int(service_time_dict[dataset.loc[from_node, "job_type"]] * seniority_multiplier[vehicle_id])

        return travel_time + service_time
    
    return time_callback

transit_callback_indices = []

for vehicle_id in range(n_vehicles):
    transit_callback = create_time_callback(vehicle_id)
    transit_callback_index = routing.RegisterTransitCallback(transit_callback)
    transit_callback_indices.append(transit_callback_index)

### Constraints

In [25]:
transit_callback_index

4

In [9]:
# Define time window contraints
time = "Time"

routing.AddDimensionWithVehicleTransits(
    transit_callback_indices,
    600,  # allow waiting time
    600,  # maximum time per vehicle
    True,  # start cumul to zero
    time,
)

time_dimension = routing.GetDimensionOrDie(time)

for location_indx in dataset.index:
    timeslot = dataset.loc[location_indx, "timeslot"]

    if timeslot is np.nan:  # no time_windows for start locations
        continue

    index = manager.NodeToIndex(location_indx)

    time_dimension.CumulVar(index).SetRange(
        time_window_dict[timeslot][0],
        time_window_dict[timeslot][1],  # window depends on promised timeslot
    )

### Objective

In [10]:
# Define cost of each arc
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

### Solution

In [11]:
# Setting heuristic strategies
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)

search_parameters.time_limit.FromSeconds(5)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)

In [12]:
solution.ObjectiveValue()

1523

### Plot Results

In [13]:
time_dimension = routing.GetDimensionOrDie(time)

for vehicle_id in range(vehicle_id):
    index = routing.Start(vehicle_id)
    plan_output = "Route for vehicle {}:\n".format(vehicle_id)
    previous_index = None

    while not routing.IsEnd(index):

        node_index = manager.IndexToNode(index)
        time_var = time_dimension.CumulVar(index)

        if previous_index is not None:
            drive_time = time_matrix[previous_index, index]
        else:
            drive_time = 0

        plan_output += "{0}::Dr {3}, Arr {1}, Job {2} -> ".format(
            node_index,
            solution.Min(time_var),
            service_time_dict[dataset.loc[node_index, "job_type"]],
            drive_time,
        )

        previous_index = index
        index = solution.Value(routing.NextVar(index))

    if allow_arbitrary_end_locations:
        plan_output += f"Schedule Finished"
    else:
        node_index = manager.IndexToNode(index)
        time_var = time_dimension.CumulVar(index)

        if previous_index is not None:
            drive_time = time_matrix[previous_index, index]
        else:
            drive_time = 0

        plan_output += "{0}::Dr {3}, Arr {1}, Job {2}".format(
            node_index,
            solution.Min(time_var),
            service_time_dict[dataset.loc[node_index, "job_type"]],
            drive_time,
        )

    plan_output += f"\nTime of the route: {solution.Min(time_var) + service_time_dict[dataset.loc[node_index, 'job_type']]}min\n"

    print(plan_output)

Route for vehicle 0:
0::Dr 0, Arr 0, Job 0 -> 18::Dr 46, Arr 46, Job 100 -> 10::Dr 33, Arr 179, Job 50 -> 9::Dr 6, Arr 235, Job 100 -> 17::Dr 116, Arr 451, Job 50 -> Schedule Finished
Time of the route: 501min

Route for vehicle 1:
2::Dr 0, Arr 0, Job 0 -> 14::Dr 20, Arr 20, Job 50 -> 15::Dr 1, Arr 81, Job 100 -> 11::Dr 25, Arr 226, Job 50 -> Schedule Finished
Time of the route: 276min

Route for vehicle 2:
3::Dr 0, Arr 0, Job 0 -> 8::Dr 51, Arr 51, Job 50 -> 1::Dr 9, Arr 240, Job 50 -> 7::Dr 16, Arr 336, Job 50 -> 16::Dr 72, Arr 488, Job 50 -> Schedule Finished
Time of the route: 538min



In [14]:
importlib.reload(f)
tours, travel_times = f.compile_tours(
    n_vehicles, routing, manager, solution, allow_arbitrary_end_locations, time_callback
)

NameError: name 'time_callback' is not defined

In [ ]:
importlib.reload(f)
fig = f.plot_tours(tours, coordinates)
fig.show()